##Solar Position algorithm

In [ ]:
  def solar_pos(self, hour, minute):
        """
        Calculate the basic solar position parameters.

        Parameters
        ----------
        hour : interger values
            value must be between 0 and 23
        minute : interger values
            value must be between 0 and 60

        Returns
        -------
        List of Equation of time, declination angle, solar hour angle,
        solar altitude and solar azimuth.

        Note
        -------
        Equations for solar position is based on the Yallops method
        (T. Muneer. Solar Radiation and Daylight Models. 2004)
        """
        loc = location()
        ut_ = (hour) + (minute/60.0) + (self.lsm/15.0)
        if int(self.mon) > 2:
            year1 = self.year
            month1 = int(self.mon) - 3
        else:
            year1 = self.year - 1
            month1 = int(self.mon) + 9
        ylt = ((ut_/24.0) + int(self.day) + int(30.6*month1+0.5) + int(365.25*
        (year1-1976)) - 8707.5)/36525.0
        termg = 357.528 + 35999.050*ylt
        if termg > 360.0:
            g360 = termg - int(termg/360.0)*360.0
        else:
            g360 = termg
        termc = 1.915*np.sin(g360*RTOD) + 0.020*np.sin(2*g360*RTOD)
        terml = 280.460 + 36000.770*ylt + termc
        if termg > 360.0:
            xl360 = terml - int(terml/360.0)*360.0
        else:
            xl360 = terml
        alpha = xl360 - 2.466*np.sin(2*xl360*RTOD) + 0.053*np.sin(4*xl360*RTOD)
        gha = 15.0*ut_-180.0 - termc + xl360-alpha
        if termg > 360.0:
            gha360 = gha - int(gha/360.0)*360.0
        else:
            gha360 = gha
        ecliptic = 23.4393 - 0.013*ylt
        eot = xl360 - termc - alpha
        dec = np.arctan(np.tan(ecliptic*RTOD)*np.sin(alpha*RTOD))/RTOD
        sha = gha360+(loc['long'])
        solelevation = np.arcsin(np.sin(loc['lat']*RTOD)*np.sin(dec*RTOD) -
        np.cos(loc['lat']*RTOD)*np.cos(dec*RTOD)*np.cos((sha+180.0)*RTOD))/RTOD
        f_1 = np.cos(loc['lat']*RTOD)*np.tan(dec*RTOD)+ \
        np.sin(loc['lat']*RTOD)*np.cos((sha+180.0)*RTOD)
        solazm = np.arccos(f_1*np.cos(dec*RTOD)/np.cos(solelevation*RTOD))/RTOD
        if sha > 0.0:
            solazm = 360.0 - solazm

        solaoi = np.arccos(np.cos(loc['Tilt']*RTOD)*np.sin(solelevation*RTOD)+\
        np.sin(loc['Tilt']*RTOD)*np.cos(solelevation*RTOD)*np.cos((solazm- \
        loc['Azimuth'])*RTOD))/RTOD

        return eot, dec, sha, solelevation, solazm, solaoi